In [ ]:
from ipywidgets import interact, widgets
interact(lambda x: x**2, x=widgets.IntSlider(min=0, max=10));

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

def scale_effects(z=0.05, omega=0.2, min_pop=10, max_pop=500, points=20):
    L_values = np.linspace(min_pop, max_pop, points)
    g_values = z * omega * L_values
    logL = np.log(L_values)

    plt.figure(figsize=(8, 5))
    plt.scatter(logL, g_values, color='purple', s=60)
    plt.plot(logL, g_values, linestyle='--', color='gray')
    plt.title("Scale Effects: log(Population) vs. Growth Rate")
    plt.xlabel("log(Population)")
    plt.ylabel("Growth Rate gY = z·ω·L")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    print("📘 SCALE EFFECTS LOGIC")
    print(f"  gY = z · ω · L → upward-sloping in log(L)")
    print(f"  This implies: bigger countries = faster growth (all else equal)")

interact(
    scale_effects,
    z=FloatSlider(value=0.05, min=0.01, max=0.2, step=0.005, description='Idea Productivity z'),
    omega=FloatSlider(value=0.2, min=0.05, max=0.9, step=0.05, description='R&D Share ω'),
    min_pop=FloatSlider(value=10, min=1, max=100, step=5, description='Min Population'),
    max_pop=FloatSlider(value=500, min=100, max=1000, step=50, description='Max Population'),
    points=FloatSlider(value=20, min=5, max=100, step=5, description='# Countries')
)

interactive(children=(FloatSlider(value=0.05, description='Idea Productivity z', max=0.2, min=0.01, step=0.005…

<function __main__.scale_effects(z=0.05, omega=0.2, min_pop=10, max_pop=500, points=20)>

# 📘 Scale Effects Hypothesis (Romer, GrowthEcon Ch. 7.4)

From the Romer model:
\[
g_Y = z \cdot \omega \cdot L
\]

- More people → more researchers → faster growth
- This predicts a **positive relationship** between \( \log(L) \) and \( g_Y \)

🔑 **Key Insight**:  
Scale effects mean large countries should grow faster — but this is debated in modern growth theory.

**Sources**:  
- GrowthEcon [Ch. 7.4](https://growthecon.com/StudyGuide/romer-scale.html)  
- Charles Jones, *Macroeconomics*, Ch. 8